In [250]:
from sklearn import preprocessing as pp
import numpy as np
from music21 import converter, instrument, note, chord, stream
import glob
import pickle
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.layers import Lambda
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from numpy.random import multinomial as randm

# Current Counts = Current Counts + Note Length % 16 ?

def buildNotes():
    
    print("In buildNotes()")
    
    notes = []
    noteLengths = []
    
    for file in glob.glob("Music/*.mid"):
        
        try:
            midi = converter.parse(file)    
            
        except:
            print("MIDI file %s failed to parse" % file)
            continue
            
        print("Parsing %s" % file)
        notesToParse = None
        
        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notesToParse = s2.parts[0].recurse()
            print("Instrument Parts %d" % len(s2.parts))
            
        except: # file has notes in a flat structure
            notesToParse = midi.flat.notes
            
        # For each note in notesToParse (a stream of notes)
            
        for element in notesToParse:
            
            # Interesting parameters of notes that we may want to look at here.
            
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
                noteLengths.append(str(element.quarterLength))
                
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
                noteLengths.append(str(element.quarterLength))

    pickle.dump(notes, open('notes.p', 'wb'))

    return notes, noteLengths

def prepareSeq(notes, noteLengths):

    seqLength = 4
    inputSize = len(notes) - seqLength
    categories = 2
    
    pitchSet = sorted(set(notes))
    npitch = len(pitchSet)
    noteToInt = dict((note, number) for number, note in enumerate(pitchSet))
    
    lengthSet = sorted(set(noteLengths))
    nlengths = len(lengthSet)
    lengthToInt = dict((length, number) for number, length in enumerate(lengthSet))
    
    # We only have 2 dimensions at the moment, pitch and length
    # noteIn - a collection of categorical arrays, each row is one category
    # noteOut - the integer which converts to the index of that category's respective
    # one hot encoding.  These categorical one hot encodings are later concatenated
    # columnwise to form a multi hot encoding.

    noteIn = [
        [],
        []
    ]
    
    noteOut = [
        [],
        []
    ]

    # Mapping of seqLength notes to note i + seqLength
    
    # X[0] sequence_in : [seqLength]
    # X[1] length_in : [seqLength]
    
    # Y[0] sequence_out : 1
    # Y[1] length_out : 1
    
    for i in range(0, inputSize, 1):
        
        sequence_in = notes[i:i + seqLength]
        sequence_out = notes[i + seqLength]
        
        length_in = noteLengths[i:i + seqLength]
        length_out = noteLengths[i + seqLength]
        
        noteIn[0].append([noteToInt[char] for char in sequence_in])
        noteIn[1].append([lengthToInt[char] for char in length_in])
        
        noteOut[0].append(noteToInt[sequence_out])
        noteOut[1].append(lengthToInt[length_out])
        
    # npatterns = len(noteIn) (inputSize)

    # reshape the input into a format compatible with LSTM layers
    # noteIn = np.reshape(noteIn, (inputSize, seqLength, 1))
    
    noteIn = np.array(noteIn, dtype=float)
    noteOut = np.array(noteOut, dtype=float)
    
    # normalize input
    noteIn[0] = noteIn[0] / float(npitch)
    noteIn[1] = noteIn[1] / float(nlengths)
    
    print("noteIn shape:")
    print(noteIn.shape)
    print("noteOut shape:")
    print(noteOut.shape)
    print(noteIn)
    
    catNote = np_utils.to_categorical(noteOut[0])
    catLength = np_utils.to_categorical(noteOut[1])
    
    # Concatenate the input arrays row wise
    # Concatenate the categorical arrays along axis 1 (columnwise)
    
    print("noteIn:")
    print(noteIn)
    
    reshape = []
    for N in range(inputSize):
        for S in range(seqLength):
            for C in range(categories):
                reshape.append(noteIn[C][N][S])
    
    dout = np.concatenate((catNote, catLength), 1)
    din = np.reshape(reshape, (inputSize, seqLength, categories))
    
    print("din shape:")
    print(din.shape)
    print("dout shape:")
    print(dout.shape)
    
    print(len(catNote[0]))
    print(len(catLength[0]))
    
    print(din[0][0][0])
    print(din[0][0][1])

    return (din, dout)


In [249]:
# print(np.arange(10).reshape(5,2).T.flatten())
notes, noteLengths = buildNotes()
noteIn, noteOut = prepareSeq(notes, noteLengths)
print(noteIn.shape)
print(noteOut.shape)

In buildNotes()
Parsing Music\Fugue1.mid
noteIn shape:
(2, 736, 4)
noteOut shape:
(2, 736)
[[[0.46511628 0.55813953 0.65116279 0.81395349]
  [0.55813953 0.65116279 0.81395349 0.95348837]
  [0.65116279 0.81395349 0.95348837 0.81395349]
  ...
  [0.97674419 0.1627907  0.8372093  0.58139535]
  [0.1627907  0.8372093  0.58139535 0.34883721]
  [0.8372093  0.58139535 0.34883721 0.51162791]]

 [[0.13333333 0.13333333 0.13333333 0.2       ]
  [0.13333333 0.13333333 0.2        0.        ]
  [0.13333333 0.2        0.         0.        ]
  ...
  [0.06666667 0.8        0.06666667 0.13333333]
  [0.8        0.06666667 0.13333333 0.06666667]
  [0.06666667 0.13333333 0.06666667 0.66666667]]]
[[[0.46511628 0.55813953 0.65116279 0.81395349]
  [0.55813953 0.65116279 0.81395349 0.95348837]
  [0.65116279 0.81395349 0.95348837 0.81395349]
  ...
  [0.97674419 0.1627907  0.8372093  0.58139535]
  [0.1627907  0.8372093  0.58139535 0.34883721]
  [0.8372093  0.58139535 0.34883721 0.51162791]]

 [[0.13333333 0.13333

In [227]:
def createNet(noteIn, nvocab):

    print("In createNet()")
    
    print(noteIn.shape)
    
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(noteIn.shape[1], noteIn.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    
    # Chris's code
    model.add(LSTM(
        512,
        input_shape=(noteIn.shape[1], noteIn.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=False
    ))
    
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation(activation='relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    
    #model.add(Flatten())
    
    model.add(Dense(nvocab))
    model.add(Lambda(lambda x: x / 0.6))
    model.add(Activation(activation='softmax'))

    model.compile(
            loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy']
        )
    
    return model

def trainNet(_epochs=1):
    """ Train a Neural Network to generate music """
    notes, noteLengths = buildNotes()

    nvocab = len(set(notes)) + len(set(noteLengths))
    
    noteIn, noteOut = prepareSeq(notes, noteLengths)
    
    model = createNet(noteIn, nvocab)
 
    checkpoint = ModelCheckpoint(
        "weights2-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5",
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )

    callbacks_list = [checkpoint]

    # Your line of code here
    model.fit(noteIn, noteOut, batch_size=64, epochs=_epochs, verbose=1, callbacks=callbacks_list)
    
def prepareSeqPred(notes, pitchnames, nvocab):
    print("In Prepare Sequences Prediction")
    noteToInt = dict((note, number) for number, note in enumerate(pitchnames))
    seqLength = 4
    noteIn = []
    output = []
    for i in range(0, len(notes) - seqLength, 1):
        sequence_in = notes[i:i + seqLength]
        sequence_out = notes[i + seqLength]
        noteIn.append([noteToInt[char] for char in sequence_in])
        output.append(noteToInt[sequence_out])

    npatterns = len(noteIn)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = np.reshape(noteIn, (npatterns, seqLength, 1))
    # normalize input
    normalized_input = normalized_input / float(nvocab)

    return (noteIn, normalized_input)

def genNotes(model, noteIn, pitchnames, nvocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # Starts the melody by picking a random sequence from the input as a starting point
    start = np.random.randint(0, len(noteIn)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = noteIn[start]
    predOut = []
    
    print("In Generate Notes: noteIn[start] = %s" % pattern)

    for note_index in range(200):
        
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(nvocab)
        
        ### Complete the line below
        prediction = model.predict(np.array(prediction_input))
        
        array = randm(1, prediction[0])
        index = np.argmax(array)
        
        result = int_to_note[index]
        predOut.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return predOut

def generate():
    print("In Generate")
    notes = pickle.load(open('notes.p', 'rb'))
    pitchnames = sorted(set(item for item in notes))
    nvocab = len(set(notes))

    noteIn, normalized_input = prepareSeqPred(notes, pitchnames, nvocab)
#    model = createNet(normalized_input, nvocab)
    
    ### Add a line to load the weights here
    model = load_model("weights2-improvement-01-4.0703-bigger.hdf5")
    
    predOut = genNotes(model, noteIn, pitchnames, nvocab)
    createMidi(predOut)
    
def createMidi(predOut):
    print("In Create Midi")
    offset = 0
    output_notes = []
    for pattern in predOut:
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        offset += 0.5
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='test_output.mid')

In [228]:
trainNet()

In buildNotes()
Parsing Music\Fugue1.mid
noteIn shape:
(2, 736, 4)
noteOut shape:
(2, 736)
din shape:
(736, 4, 2)
dout shape:
(736, 58)
43
15
0
0
In createNet()
(736, 4, 2)
Epoch 1/1
736/736 [==============================] - 10s 13ms/step - loss: 8.0140 - acc: 0.0014


In [229]:
generate()

In Generate
In Prepare Sequences Prediction
In Generate Notes: noteIn[start] = [41, 10, 29, 36]


KeyError: 77